In [1]:
using CausalELM
using CSV
using DataFrames
using Random

In [2]:
pension_df = CSV.read("pension.csv", DataFrame)
pension_df = pension_df[:, [10, 22, 13, 14, 15, 18, 20, 17, 24, 33]]
covariates, treatment, outcome = pension_df[:, 3:end], pension_df[:, 2], pension_df[:, 1]

(9915×8 DataFrame
  Row │ age    inc    fsize  marr   twoearn  db     pira   hown  
      │ Int64  Int64  Int64  Int64  Int64    Int64  Int64  Int64 
──────┼──────────────────────────────────────────────────────────
    1 │    31  28146      5      1        0      0      0      1
    2 │    52  32634      5      0        0      0      0      1
    3 │    50  52206      3      1        1      0      1      1
    4 │    28  45252      4      1        1      0      0      0
    5 │    42  33126      3      0        0      1      0      1
    6 │    49  76860      6      1        1      1      0      1
    7 │    40  57477      4      1        1      1      0      1
    8 │    58  14637      1      0        0      0      0      0
  ⋮   │   ⋮      ⋮      ⋮      ⋮       ⋮       ⋮      ⋮      ⋮
 9909 │    28  31926      2      1        1      0      0      0
 9910 │    49  64215      4      1        1      0      1      1
 9911 │    34  13500      1      0        0      1      0      0
 9912 

In [3]:
covariates.inc = (covariates.inc .- minimum(covariates.inc))/(maximum(covariates.inc) - minimum(covariates.inc))
covariates.age = (covariates.age .- minimum(covariates.age))/(maximum(covariates.age) - minimum(covariates.age))
covariates.fsize = (covariates.fsize .- minimum(covariates.fsize))/(maximum(covariates.fsize) - minimum(covariates.fsize))
covariates.marr = (covariates.marr .- minimum(covariates.marr))/(maximum(covariates.marr) - minimum(covariates.marr))
covariates

Row,age,inc,fsize,marr,twoearn,db,pira,hown
,Float64,Float64,Float64,Float64,Int64,Int64,Int64,Int64
1,0.153846,0.125821,0.333333,1.0,0,0,0,1
2,0.692308,0.144156,0.333333,0.0,0,0,0,1
3,0.641026,0.224115,0.166667,1.0,1,0,1,1
4,0.0769231,0.195705,0.25,1.0,1,0,0,0
5,0.435897,0.146166,0.166667,0.0,0,1,0,1
6,0.615385,0.324836,0.416667,1.0,1,1,0,1
7,0.384615,0.245649,0.25,1.0,1,1,0,1
8,0.846154,0.0706319,0.0,0.0,0,0,0,0
9,0.102564,0.0376875,0.25,0.0,0,0,0,0


In [4]:
dr_learner = DoubleMachineLearning(covariates, treatment, outcome, num_feats=6)

DoubleMachineLearning([0.15384615384615385 0.1258211589371507 … 0.0 1.0; 0.6923076923076923 0.1441562898323365 … 0.0 1.0; … ; 0.41025641025641024 0.24039121482498285 … 0.0 1.0; 0.07692307692307693 0.11789145994705363 … 0.0 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [-3300.0, 61010.0, 8849.0, -6013.0, -2375.0, -11000.0, -16901.0, 1000.0, 0.0, 6400.0  …  -1436.0, 4500.0, 34739.0, -750.0, 40000.0, 172.0, 836.0, 6150.0, 14499.0, -5400.0], "ATE", false, "regression", CausalELM.relu, 9915, 100, 6, 24, NaN, 5)

In [12]:
estimate_causal_effect!(dr_learner)

8667.309064475481

In [9]:
summarize(dr_learner)

Dict{Any, Any} with 11 entries:
  "Activation Function"    => relu
  "Quantity of Interest"   => "ATE"
  "Sample Size"            => 9915
  "Number of Machines"     => 100
  "Causal Effect"          => 8806.5
  "Number of Neurons"      => 24
  "Task"                   => "regression"
  "Time Series/Panel Data" => false
  "Standard Error"         => NaN
  "p-value"                => NaN
  "Number of Features"     => 6

In [10]:
validate(dr_learner)

(Dict(0.025 => -12979.904119051262, 0.075 => -12217.068316708708, 0.1 => -6143.33640640303, 0.05 => -9062.747974951273), 2.8344920146887382, Matrix{Float64}(undef, 0, 9))